In [1]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

# settings
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')

interval_comment_whole = "WholeData"
start_time_whole = pd.to_datetime('2014-03-24')
end_time_whole = pd.to_datetime('2015-08-12')

suffix = "_5sec_"

infile_jobs = datadir + "CPUs/" + interval_comment + "/jobs_cleaned.csv"

## notice that 43 doesn't exists in the db
nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
# cpus=['02'] # test


In [3]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_data = pd.read_csv(infile_jobs)
print("jobs_whole_data contains {} records".format(jobs_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records
jobs_whole_data contains 404792 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
interval_data_jobs = jobs_data[pd.to_datetime(jobs_data['run_start_time']) >= start_time]
interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs['end_time']) <= end_time]
print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))

interval_data_jobs contains 100003 records


#### solve the problem of jobs runned on node 129

In [6]:
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

after the clean, jobs_to_nodes_whole_data contains 444610 records


In [8]:
### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(interval_data_jobs, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

merged_jobs_to_nodes contains 108074 records


In [9]:
# drop the jobs out of time range
#interval_data_jobs = merged_jobs_to_nodes[pd.to_datetime(merged_jobs_to_nodes['run_start_time']) >= start_time]
#interval_data_jobs = interval_data_jobs[pd.to_datetime(interval_data_jobs['end_time']) <= end_time]
#interval_data_jobs = interval_data_jobs.sort_values('run_start_time')
#print("interval_data_jobs contains {} records".format(interval_data_jobs.shape[0]))

In [9]:
merged_jobs_to_nodes.head(1)

,job_id,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,...,dependency,exit_status,pbs_exit_code,gpu_req,mic_req,node_id,ncpus,ngpus,nmics,mem_requested
0,1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,...,NaN,COMPLETED,NaN,NaN,NaN,17,16,0,0,14680064


drop where node_id is NaN

torna l'assenza del node 43 e la scarsità di 22, 41, 53,

In [10]:
#regular_data_jobs = merged_jobs_to_nodes[merged_jobs_to_nodes.node_id >= 1]
#regular_data_jobs = regular_data_jobs[regular_data_jobs.node_id <= 64]
#regular_data_jobs.shape

# final

In [13]:
for node in nodes:
    print(node)
    interval_jobs_to_node = merged_jobs_to_nodes[merged_jobs_to_nodes['node_id'] == int(node)]
    print("interval_jobs_node_data contains {} records".format(interval_jobs_to_node.shape[0]))
    # read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment
    
    node_data = pd.read_csv(infile_node + ".csv", index_col=1)
    print(node_data.shape[0])
    node_start_time = node_data.index[0]
    #node_data.reset_index()
    
    num_minutes = node_data.shape[0]
    active_cores = pd.Series(0, index=np.arange(num_minutes))
    active_jobs = pd.Series(0, index=np.arange(num_minutes))
    active_gpus = pd.Series(0, index=np.arange(num_minutes))
    active_mics = pd.Series(0, index=np.arange(num_minutes))
    
    i = 0
    num_jobs = interval_jobs_to_node.shape[0]
    for index, row in interval_jobs_to_node.iterrows():
        #print("{}/{}   {}".format(i, num_jobs, row['job_id_string']))
        i = i + 1
        job_start_time = pd.to_datetime(row['run_start_time'])
        # print(start_time)
        job_end_time = pd.to_datetime(row['end_time'])
        job_cores = row['ncpus']
        job_gpus = row['ngpus']
        job_mics = row['nmics']
        
        #print(job_start_time)
        #print(pd.to_datetime(node_start_time))
        before_minutes = int((job_start_time - pd.to_datetime(node_start_time)) / np.timedelta64(5, 's'))
        running_minutes = int((job_end_time - job_start_time) / np.timedelta64(5, 's'))
        after_minutes = num_minutes - running_minutes - before_minutes
        #print("{} - {} - {}".format(before_minutes, running_minutes, after_minutes))
        before_serie = pd.Series(0, index=np.arange(before_minutes))
        after_serie = pd.Series(0, index=np.arange(after_minutes))
        running_serie = pd.Series(job_cores, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_cores = active_cores.add(concat_series, fill_value=0)

        running_serie = pd.Series(1, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_jobs= active_jobs.add(concat_series, fill_value=0)

        running_serie = pd.Series(job_gpus, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_gpus = active_gpus.add(concat_series, fill_value=0)

        running_serie = pd.Series(job_mics, index=np.arange(running_minutes))
        concat_series = pd.concat([before_serie, running_serie, after_serie], ignore_index=True)
        active_mics = active_mics.add(concat_series, fill_value=0)
        
    node_data['active_cores'] = active_cores.values
    node_data['active_jobs'] = active_jobs.values
    node_data['active_gpus'] = active_gpus.values
    node_data['active_mics'] = active_mics.values
    
    node_data.to_csv(infile_node + "_active_cores_and_jobs" + ".csv")

01
interval_jobs_node_data contains 555 records


FileNotFoundError: [Errno 2] File b'/datasets/eurora_data/db1/CPUs/AfterAlina/node01_5sec_AfterAlina.csv' does not exist: b'/datasets/eurora_data/db1/CPUs/AfterAlina/node01_5sec_AfterAlina.csv'